In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# Dateien einlesen
idn_df = pd.read_excel("idn.xlsx", dtype=str)  # IDNs laden
keywords_df = pd.read_excel("Schlagwort.xlsx", dtype=str)  # Schlagwörter laden

# Schlagwörter in eine Liste umwandeln
keywords = keywords_df.iloc[:, 0].dropna().tolist()

# Funktion zum Abrufen und Durchsuchen der Texte
def find_keywords(idn):
    url_text = f"https://d-nb.info/{idn}/04/text"
    url_pdf = f"https://d-nb.info/{idn}/04/pdf"
    try:
        response = requests.get(url_text, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            text = soup.get_text().lower()  # Ganze Seite als Text
            
            found_words = []
            for word in keywords:
                count = len(re.findall(rf"\b{re.escape(word.lower())}\b", text))  # Zählt exakte Wortvorkommen
                if count > 0:
                    found_words.append(f"{word} ({count})")  # Speichert das Wort + Anzahl
            
            return url_text, url_pdf, ", ".join(found_words) if found_words else None  # Falls kein Treffer, None
    except requests.RequestException:
        return url_text, url_pdf, "Fehler"
    return url_text, url_pdf, None  # Falls kein Schlagwort gefunden, Rückgabe als None

# Neue Spalten füllen
idn_df[["URL", "PDF-URL", "Gefundenes Schlagwort"]] = idn_df.iloc[:, 0].apply(find_keywords).apply(pd.Series)

# Nur Zeilen mit gefundenen Schlagwörtern behalten
idn_df = idn_df.dropna(subset=["Gefundenes Schlagwort"])

# Spaltennamen setzen
idn_df.rename(columns={idn_df.columns[0]: "IDN"}, inplace=True)

# Ergebnis speichern, nur wenn es Treffer gibt
if not idn_df.empty:
    idn_df.to_excel("idn_mit_schlagwort.xlsx", index=False)
    print("Die Datei 'idn_mit_schlagwort.xlsx' wurde erfolgreich erstellt!")
else:
    print("Keine Schlagwörter gefunden – es wurde keine Datei erstellt.")


/home/dbsm/.local/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Die Datei 'idn_mit_schlagwort.xlsx' wurde erfolgreich erstellt!
